<a href="https://colab.research.google.com/github/animalran/csvfiles/blob/master/Backtest_youtube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install empyrical
import empyrical
import pandas as pd
from empyrical import max_drawdown, alpha_beta, cum_returns

import math
import statistics as stat
def load_equities_web(symbol, date_from):
    import pandas_datareader.data as web
    raw_data = web.DataReader(symbol, 'yahoo', pd.to_datetime(date_from), pd.datetime.now())
    data = raw_data.stack(dropna=False)['Adj Close'].to_frame().reset_index().rename(columns = {'Symbols':'symbol', 'Date':'date', 'Adj Close':'value'}).sort_values(by = ['symbol', 'date'])
    return pd.pivot_table(data, columns = 'symbol', index = 'date', values ='value')


prices = load_equities_web(['SPY', '^GSPC', '^VIX'], date_from = '2000-01-01')

def backtest_strategy(prices, symbol_trade, symbol_volatility, volatility_threshold, capital, symbol_benchmark):
    
    
    df_init   = (prices[symbol_trade]*0).to_frame().assign(cash = 0)
    df_update = (prices[symbol_trade]*0).to_frame().assign(cash = 0)
    df_end    = (prices[symbol_trade]*0).to_frame().assign(cash = 0)
    #print(df_init.head(5))
    df_init.iloc[0, df_init.columns.get_loc('cash')] = capital
    df_end.iloc[0, df_end.columns.get_loc('cash')]   = capital
    #print(df_init.head(5))
    calendar = pd.Series(prices.index).iloc[1:]
    
    
    for date in calendar:
        prev_date = df_init.index[df_init.index<date][-1]    
        df_init.loc[date, :] = df_end.loc[prev_date, :]
        port_value = df_init.loc[date, symbol_trade] * prices.loc[date, symbol_trade] + df_init.loc[date, 'cash']
        #print(port_value)
        #print(prices.loc[date, symbol_trade]
        if prices.loc[date, symbol_volatility] > volatility_threshold: # volatility is high -> be fully in cash
            df_end.loc[date, symbol_trade] = 0
            df_end.loc[date, 'cash']       = port_value
        else: # volatility is low -> be in market position
            df_end.loc[date, symbol_trade] = port_value/prices.loc[date, symbol_trade]
            df_end.loc[date, 'cash'] = 0
        df_update.loc[date] = df_end.loc[date] - df_init.loc[date]
    #print(df_update.tail(5))
    #print(df_init.tail(5))
    portval = (df_end*prices.assign(cash = 1)[[symbol_trade, 'cash']]).sum(axis = 1).to_frame().rename(columns = {0:'strategy'})
    #print(portval)
    portval['benchmark'] = prices[symbol_benchmark]
    #print(portval)
    portval = portval/portval.iloc[0].values
    dr  =    portval['strategy'].pct_change(1)
    dr  =   dr.iloc[1:]
    rf  =   .06
    td  =   252
    ex_dr   =   dr-rf/td
    l   =   math.sqrt(td)
    m   =   stat.mean(ex_dr)
    o   =   stat.stdev(ex_dr)
    sharpe  =   l*m/o
    print(sharpe)
    return portval
    cr=empyrical.cum_returns(dr)
    print("Maximum Draw Down :"+str(empyrical.max_drawdown(dr)))
    print("Annual Returns: "+str(empyrical.annual_return(dr, period='daily', annualization=None)))
    print("CAGR : "+str(empyrical.cagr(dr, period='daily', annualization=None)))
    print("Sharpe Ratio : "+str(empyrical.sharpe_ratio(dr, risk_free=0.06/252)))
    print("Tail Ratio : "+str(empyrical.tail_ratio(dr)))
    print("Time Series Stability : "+str(empyrical.stability_of_timeseries(dr)))
    print("Sortino Ration: "+str(empyrical.sortino_ratio(dr, required_return=10)))
    

res = backtest_strategy(prices = prices, symbol_trade = 'SPY', symbol_volatility = '^VIX', volatility_threshold = 20, capital = 10000, symbol_benchmark = '^GSPC')
print(res)
#print(prices)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.
  # Remove the CWD from sys.path while we load stuff.


-0.24169436386194387
            strategy  benchmark
date                           
2000-01-03  1.000000   1.000000
2000-01-04  1.000000   0.961655
2000-01-05  1.000000   0.963504
2000-01-06  1.000000   0.964425
2000-01-07  1.000000   0.990551
...              ...        ...
2020-06-29  2.001701   2.098130
2020-06-30  2.001701   2.130461
2020-07-01  2.001701   2.141161
2020-07-02  2.001701   2.150884
2020-07-06  2.001701   2.185044

[5159 rows x 2 columns]
